首先 import 必要的模块

In [1]:
import lightgbm as lgbm
#from lgbm.sklearn import LGBMClassifier
import xlearn as xl
import pandas as pd 
import numpy as np
import time

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss     #采用logloss作为评价指标
from pandas import DataFrame

from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder
from matplotlib import pyplot
import seaborn as sns
%matplotlib inline

## 1.1读取数据

In [2]:
train_name_1='400_0'
train_name_2='400_1'
valid_name_1='400_2'
valid_name_2='400_3'

In [48]:
#train_data
train_1 = pd.read_csv("/home/jupyter/训练数据组/400w数据组/"+train_name_1,index_col=0).reset_index(drop=True)   #拿200万条数据训练
train_1.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(train_1.shape)
#validation_data
valid_1 = pd.read_csv("/home/jupyter/训练数据组/400w数据组/"+valid_name_1,index_col=0).reset_index(drop=True)   #拿200万条数据训练
valid_1.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(valid_1.shape)

(3996246, 23)
(3886316, 23)


In [30]:
#train_data
train_2 = pd.read_csv("/home/jupyter/训练数据组/100w数据组/"+train_name_2,index_col=0).reset_index(drop=True)   #拿200万条数据训练
train_2.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(train_2.shape)
valid_2 = pd.read_csv("/home/jupyter/训练数据组/100w数据组/"+valid_name_2,index_col=0).reset_index(drop=True)   #拿200万条数据训练
valid_2.drop(['id','hour','user_id','Unnamed: 0'],axis=1,inplace=True)
print(valid_2.shape)

(999864, 23)
(996224, 23)


In [28]:
#test_data
test = pd.read_csv("/home/jupyter/训练数据组/FE_test_26_final.csv",index_col=0).reset_index(drop=True)   #拿200万条数据训练
test.drop(['id','hour','user_id'],axis=1,inplace=True)
test.shape

/home/qiaoliang/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(4218938, 23)

In [49]:
#first
y_train_1 = train_1['click']
x_train_1 = train_1.drop(["click"], axis=1)

y_valid_1 = valid_1['click']
x_valid_1 = valid_1.drop(["click"], axis=1)

In [6]:
#second
y_train_2 = train_2['click']
x_train_2 = train_2.drop(["click"], axis=1)

y_valid_2 = valid_2['click']
x_valid_2 = valid_2.drop(["click"], axis=1)

In [30]:
y_test = test['click']
x_test = test.drop(["click"], axis=1)

### 1.2 original_data generate libffm

In [50]:
#fm_data
x_train_fm_1=np.array(x_train_1)
x_valid_fm_1=np.array(x_valid_1)
x_test_fm_1=np.array(x_test)
xl.write_data_to_xlearn_format(X=x_train_fm_1,y=y_train_1,filepath=r'/home/jupyter/xlearn/fm/fm_train.txt')
xl.write_data_to_xlearn_format(X=x_valid_fm_1,y=y_valid_1,filepath=r'/home/jupyter/xlearn/fm/fm_valid.txt')

In [ ]:
xl.write_data_to_xlearn_format(X=x_test_fm_1,y=y_test,filepath=r'/home/jupyter/xlearn/fm/fm_test.txt')

In [30]:
del train_1
del train_2
del valid_1
del valid_2
del test

默认参数，此时学习率为0.1，比较大，观察弱分类数目的大致范围
（采用默认参数配置，看看模型是过拟合还是欠拟合）

## x-learn train

### 2.fm模型训练

In [6]:
# original Training task
fm_model = xl.create_fm()
fm_model.setSign()  # Convert output to 0-1  scope = xl.create_fm() # Use field-aware factorization machine
fm_model.setTrain(r'/home/jupyter/xlearn/fm/fm_train.txt')  # Training data
fm_model.setValidate(r'/home/jupyter/xlearn/fm/fm_valid.txt')  # Validation data
fm_model.setTest(r'/home/jupyter/xlearn/fm/fm_test.txt')  # test data

In [7]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.00001, 
         'metric':'acc','fold':2,'epoch':25,'stop_window':3}

fm_model.setSign()  # Convert output to 0-1  scope
result=fm_model.fit(param, r'/home/jupyter/xlearn/fm/model.out')
result_cv=fm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)

78.06491708755493


In [8]:
time.sleep(5)
fm_model.predict(r'/home/jupyter/xlearn/fm/model.out', r'/home/jupyter/xlearn/fm/output.txt')
#0.4443,0.4456,0.4422

In [13]:
#lambda调整

In [9]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.0001, 
         'metric':'acc','fold':2,'epoch':25,'stop_window':3}

fm_model.setSign()  # Convert output to 0-1  scope
result=fm_model.fit(param, r'/home/jupyter/xlearn/fm/model.out')
result_cv=fm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)
time.sleep(5)
fm_model.predict(r'/home/jupyter/xlearn/fm/model.out', r'/home/jupyter/xlearn/fm/output.txt')
#0.4468,0.448,0.4457

94.65050840377808


In [10]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.000001, 
         'metric':'acc','fold':2,'epoch':25,'stop_window':3}

fm_model.setSign()  # Convert output to 0-1  scope
result=fm_model.fit(param, r'/home/jupyter/xlearn/fm/model.out')
result_cv=fm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)
time.sleep(5)
fm_model.predict(r'/home/jupyter/xlearn/fm/model.out', r'/home/jupyter/xlearn/fm/output.txt')
#0.4468,0.448,0.4416

68.1694347858429


In [11]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.0000001, 
         'metric':'acc','fold':2,'epoch':25,'stop_window':3}

fm_model.setSign()  # Convert output to 0-1  scope
result=fm_model.fit(param, r'/home/jupyter/xlearn/fm/model.out')
result_cv=fm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)
time.sleep(5)
fm_model.predict(r'/home/jupyter/xlearn/fm/model.out', r'/home/jupyter/xlearn/fm/output.txt')
#0.4468,0.448,0.4413

66.54297685623169


In [12]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.2,'lambda':0.00000001, 
         'metric':'acc','fold':2,'epoch':25,'stop_window':3}

fm_model.setSign()  # Convert output to 0-1  scope
result=fm_model.fit(param, r'/home/jupyter/xlearn/fm/model.out')
result_cv=fm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)
time.sleep(5)
fm_model.predict(r'/home/jupyter/xlearn/fm/model.out', r'/home/jupyter/xlearn/fm/output.txt')
#0.4468,0.448,0.4413

71.16335582733154


In [14]:
#学习率调整

In [15]:
# param:
#  0. binary classification
#  1. learning rate: 0.2
#  2. regular lambda: 0.002
#  3. evaluation metric: accuracy
start=time.time()
param = {'task':'binary','k':4,'opt':'adagrad','lr':0.5,'lambda':0.00000001, 
         'metric':'acc','fold':2,'epoch':25,'stop_window':3}

fm_model.setSign()  # Convert output to 0-1  scope
result=fm_model.fit(param, r'/home/jupyter/xlearn/fm/model.out')
result_cv=fm_model.cv(param)
# Prediction task
end=time.time()
print(end-start)
time.sleep(5)
fm_model.predict(r'/home/jupyter/xlearn/fm/model.out', r'/home/jupyter/xlearn/fm/output.txt')
#0.4468,0.448,0.4413

81.30421829223633
